<a href="https://colab.research.google.com/github/ganeshkumar269/colabnotebooks/blob/main/miniproject_efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import pickle
import csv

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet,ResNet50
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K

In [ ]:
!pip install -U efficientnet

Requirement already up-to-date: efficientnet in /usr/local/lib/python3.6/dist-packages (1.1.1)


In [5]:
K.set_learning_phase(1)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [6]:
img_width, img_height = 224, 224
nb_train_samples = 1400
nb_validation_samples = 600
epochs = 5
batch_size = 32
n_classes = 2

In [7]:
path_base = '/content/gdrive/My Drive/'

In [8]:
train_dir = path_base + 'ISIC_2017/train/'
test_dir  = path_base + 'ISIC_2017/validate/'
train_df = pd.read_csv('/content/gdrive/My Drive/newfile.csv')
train_df.head()

,id,value
0,ISIC_0000000.jpg,0
1,ISIC_0000001.jpg,0
2,ISIC_0000002.jpg,1
3,ISIC_0000003.jpg,0
4,ISIC_0000004.jpg,1


In [9]:
train_df['value'] = train_df['value'].astype(str)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.10,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_dir,
    x_col="id",
    y_col="value",
    target_size=(32,32),
    subset="training",
    batch_size=5000,
    shuffle=True,
    class_mode="binary"
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_dir,
    x_col="id",
    y_col="value",
    target_size=(32,32),
    subset="validation",
    batch_size=2000,
    shuffle=True,
    class_mode="binary"
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 599 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 1261 validated image filenames belonging to 2 classes.
Found 140 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 599 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [11]:
test_datagen = ImageDataGenerator(
    rescale=1/255
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32,32),
    batch_size=1,
    shuffle=False,
    class_mode=None
)

Found 0 images belonging to 1 classes.


In [12]:
from keras import applications
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from keras import callbacks
from keras.models import Sequential

In [ ]:
from keras.layers import Dense
from keras.optimizers import Adam

def getResNet():
  resnet = ResNet50(
      weights='imagenet',
      input_shape=(32,32,3),
      include_top=False,
      pooling='max'
  )

  model = Sequential()
  model.add(resnet)
  model.add(Dense(units = 120, activation='relu'))
  model.add(Dense(units = 120, activation = 'relu'))
  model.add(Dense(units = 1, activation='sigmoid'))
  return model


In [ ]:
from keras.layers import Dense
from keras.optimizers import Adam

def getEfficientNet():
  efficient_net = EfficientNetB3(
      weights='imagenet',
      input_shape=(32,32,3),
      include_top=False,
      pooling='max'
  )

  model = Sequential()
  model.add(efficient_net)
  model.add(Dense(units = 120, activation='relu'))
  model.add(Dense(units = 120, activation = 'relu'))
  model.add(Dense(units = 1, activation='sigmoid'))
  return model

In [14]:
model = getResNet()
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs = 50,
    steps_per_epoch = 1,
    validation_data = validation_generator,
    validation_steps = 7
)